# Test the Fine-Tuned Model

If you do not have a GPU, you can test the fine-tuned model here in Google Colab with the provided free T4 GPU.

#### Imports

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

#### Load fine-tuned model
I use the optimized inference function from [unsloth](https://github.com/unslothai/unsloth) which is faster than the Hugging Face inference pipeline.

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "jost/mistral7b_plantuml",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit)

#### Prompt
I prompt the model with an example from the dataset and format it properly. The line breaks are important in the description. If you omit them, the model will also output PlantUML code without line breaks.

In [ ]:
prompt ="""\nFor the given description, generate \na Sequence diagram diagram using plantuml. \nDescription: Use Case Name: Patient Registration\nUse Case ID: HC-001\n\nUse Case Description:\nThis use case describes the process of registering a new patient in a healthcare system.\n\nUse Case Actors:\n1. Front desk staff\n2. Patient\n\nUse Case Triggers:\n- A new patient arrives at the healthcare facility and wants to register.\n\nUse Case Preconditions:\n- The patient has not been registered in the system before.\n- The front desk staff is available to assist the patient.\n\nUse Case Postconditions:\n- The patient\'s information is recorded in the healthcare system.\n- The patient is assigned a unique identification number.\n\nUse Case Flow:\n1. The patient approaches the front desk and expresses the intention to register.\n2. The front desk staff welcomes the patient and requests basic information such as name, date of birth, address, contact number, and insurance details.\n3. The front desk staff enters the patient\'s information into the healthcare system.\n4. The system validates the entered information for accuracy.\n5. If any errors are detected, the front desk staff corrects the information.\n6. Once the information is accurate, the system generates a unique patient identification number.\n7. The front desk staff assigns the identification number to the patient and provides any necessary registration documents.\n8. The front desk staff informs the patient about the next steps, such as appointment scheduling or waiting for a healthcare provider.\n9. The patient thanks the front desk staff and proceeds accordingly.\n\nExceptions:\n- If the patient\'s information already exists in the system, the front desk staff alerts the patient and follows the appropriate process for updating the existing record.\n- If there are technical issues with the healthcare system, the front desk staff uses an alternative method for recording the patient\'s information, such as paper forms.\n\nBusiness Rules:\n- The patient\'s information must be accurate and up-to-date.\n- The patient must provide valid identification and insurance details, if applicable.\n- The patient\'s privacy and confidentiality must be maintained throughout the registration process.\n\nNotes:\n- This use case focuses on the initial registration of a new patient and does not cover subsequent visits or updates to the patient\'s information.\n- The specific fields and information required may vary depending on the healthcare facility and its policies.\n\nScenarios:\n1. The patient provides incorrect contact information during registration:\n   - The front desk staff identifies the error and asks the patient to provide the correct information.\n   - The staff updates the patient\'s contact details in the system and re-verifies the accuracy.\n\n2. The healthcare system experiences a temporary outage during registration:\n   - The front desk staff informs the patient about the technical issue and apologizes for the inconvenience.\n   - The staff records the patient\'s information manually using paper forms until the system is restored.\n   - Once the system is back online, the staff transfers the manually recorded information into the system.\n\nAdd import elements which are missing in the description to\ngenerate a complete diagram.\n\nThe basic syntax for the diagram is as follows:\n@startuml\nparticipant User as U\nparticipant "Order System" as OS\nparticipant "Payment System" as PS\nU -> OS: Place Order\nactivate OS\nOS -> PS: Request Payment\nactivate PS\nPS -->> U: Payment Confirmation\ndeactivate PS\nOS -->> U: Order Confirmation\ndeactivate OS\n@enduml\n"""
template = f"[INST] {prompt} [/INST]"

#### Run inference
The inference function returns the prompt as well. I remove the prompt and the special tokens from the generated output to get the generated PlantUML code.

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer([template], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True)
response = tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
generated_response = response[0].replace(template, "")
print(generated_response[4:-4])

@startuml
actor "Front Desk Staff" as FDS
actor Patient as P

participant "Healthcare System" as HS

FDS -> P: Welcome and request basic information
activate P
P -> FDS: Provide basic information
deactivate P
FDS -> HS: Enter patient information
activate HS
HS -> FDS: Validate information
deactivate HS
FDS -> HS: Correct information if necessary
activate HS
HS -> FDS: Generate unique patient identification number
deactivate HS
FDS -> P: Assign identification number and provide registration documents
activate P
P -> FDS: Thank and proceed accordingly
deactivate P
FDS -> HS: Inform patient about next steps
activate HS
HS -> FDS: Provide next steps information
deactivate HS
FDS -> P: Inform patient about next steps
activate P
P -> FDS: Thank and proceed accordingly
deactivate P

note left of FDS: Exceptions
note right of FDS: Business Rules
note over FDS: Scenarios

@enduml
